In [1]:
from haystack import Document, Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.classifiers import DocumentLanguageClassifier
from haystack.components.routers import MetadataRouter
from haystack.components.writers import DocumentWriter

In [2]:
documents = [
    Document(content = "Durchgängig Lärm, weil direkt an der Partymeile; schmutziges Geschirr; unvollständige Küchenausstattung; Abzugshaube über Herd ging für zwei Stunden automatisch an und lies sich nicht abstellen; Reaktionen auf Anfragen entweder gar nicht oder unfreundlich"),
    Document(content = "Das Personal ist sehr zuvorkommend! Über WhatsApp war man im guten Kontakt und konnte alles erfragen. Auch das Angebot des Shuttleservices war super und würde ich empfehlen - sehr unkompliziert! Unser Flug hatte Verspätung und der Shuttle hat auf uns gewartet. Die Lage zur Innenstadt ist sehr gut,jedoch ist die Fensterfront direkt zur Club-Straße deshalb war es nachts bis drei/vier Uhr immer recht laut. Die Kaffeemaschine oder auch die Couch hätten sauberer sein können. Ansonsten war das Appartement aber völlig ok."),
    Document(content = "Super appartement. Juste au dessus de plusieurs bars qui ferment très tard. A savoir à l'avance. (Bouchons d'oreilles fournis !)"),
    Document(content = "El apartamento estaba genial y muy céntrico, todo a mano. Al lado de la librería Lello y De la Torre de los clérigos. Está situado en una zona de marcha, así que si vais en fin de semana , habrá ruido, aunque a nosotros no nos molestaba para dormir"),
    Document(content = "The keypad with a code is convenient and the location is convenient. Basically everything else, very noisy, wi-fi didn't work, check-in person didn't explain anything about facilities, shower head was broken, there's no cleaning and everything else one may need is charged."),
    Document(content = "It is very central and appartement has a nice appearance (even though a lot IKEA stuff), *W A R N I N G** the appartement presents itself as a elegant and as a place to relax, very wrong place to relax - you cannot sleep in this appartement, even the beds are vibrating from the bass of the clubs in the same building - you get ear plugs from the hotel -> now I understand why -> I missed a trip as it was so loud and I could not hear the alarm next day due to the ear plugs.- there is a green light indicating 'emergency exit' just above the bed, which shines very bright at night - during the arrival process, you felt the urge of the agent to leave as soon as possible. - try to go to 'RVA clerigos appartements' -> same price, super quiet, beautiful, city center and very nice staff (not an agency)- you are basically sleeping next to the fridge, which makes a lot of noise, when the compressor is running -> had to switch it off - but then had no cool food and drinks. - the bed was somehow broken down - the wooden part behind the bed was almost falling appart and some hooks were broken before- when the neighbour room is cooking you hear the fan very loud. I initially thought that I somehow activated the kitchen fan"),
    Document(content = "Un peu salé surtout le sol. Manque de service et de souplesse"),
    Document(content = "De comfort zo centraal voor die prijs."),
    Document(content = "Die Lage war sehr Zentral und man konnte alles sehenswertes zu Fuß erreichen. Wer am Wochenende nachts schlafen möchte, sollte diese Unterkunft auf keinen Fall nehmen. Party direkt vor der Tür so das man denkt, man schläft mitten drin. Sehr Sehr laut also und das bis früh 5 Uhr. Ab 7 kommt dann die Straßenreinigung die keineswegs leiser ist."),
    Document(content = "Nous avons passé un séjour formidable. Merci aux personnes , le bonjours à Ricardo notre taxi man, très sympathique. Je pense refaire un séjour parmi vous, après le confinement, tout était parfait, surtout leur gentillesse, aucune chaude négative. Je n'ai rien à redire de négative, Ils étaient a notre écoute, un gentil message tout les matins, pour nous demander si nous avions besoins de renseignement et savoir si tout allait bien pendant notre séjour."),
    Document(content = "Céntrico. Muy cómodo para moverse y ver Oporto. Edificio con terraza propia en la última planta. Todo reformado y nuevo. Te traen un estupendo desayuno todas las mañanas al apartamento. Solo que se puede escuchar algo de ruido de la calle a primeras horas de la noche. Es un zona de ocio nocturno. Pero respetan los horarios.")
]

In [3]:
fr_document_store = InMemoryDocumentStore()
en_document_store = InMemoryDocumentStore()
de_document_store = InMemoryDocumentStore()
es_document_store = InMemoryDocumentStore()
nl_document_store = InMemoryDocumentStore()

In [4]:
language_classifier = DocumentLanguageClassifier(languages=["fr", "en", "de", "es", "nl"])

router_rules = {
    "en": {"language": {"$eq": "en"}},
    "fr": {"language": {"$eq": "fr"}},
    "de": {"language": {"$eq": "de"}},
    "es": {"language": {"$eq": "es"}},
    "nl": {"language": {"$eq": "nl"}}
}
router = MetadataRouter(rules=router_rules)

In [5]:
fr_writer = DocumentWriter(document_store=fr_document_store)
en_writer = DocumentWriter(document_store=en_document_store)
de_writer = DocumentWriter(document_store=de_document_store)
es_writer = DocumentWriter(document_store=es_document_store)
nl_writer = DocumentWriter(document_store=nl_document_store)

In [6]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_component("language_classifier", language_classifier)
indexing_pipeline.add_component("router", router)
indexing_pipeline.add_component("fr_writer", fr_writer)
indexing_pipeline.add_component("en_writer", en_writer)
indexing_pipeline.add_component("de_writer", de_writer)
indexing_pipeline.add_component("es_writer", es_writer)
indexing_pipeline.add_component("nl_writer", nl_writer)

In [7]:
indexing_pipeline.connect("language_classifier", "router")
indexing_pipeline.connect("router.fr", "fr_writer")
indexing_pipeline.connect("router.en", "en_writer")
indexing_pipeline.connect("router.de", "de_writer")
indexing_pipeline.connect("router.es", "es_writer")
indexing_pipeline.connect("router.nl", "nl_writer")

In [8]:
indexing_pipeline.run({"language_classifier": {"documents": documents}})

{'router': {'unmatched': []},
 'en_writer': {'documents_written': 2},
 'fr_writer': {'documents_written': 3},
 'de_writer': {'documents_written': 3},
 'es_writer': {'documents_written': 2},
 'nl_writer': {'documents_written': 1}}

In [9]:
en_document_store.filter_documents()

[Document(id=8f64ab234c6a5d5652d02bed144d069ec6e988903b071d16fffbf400abfc1047, content: 'The keypad with a code is convenient and the location is convenient. Basically everything else, very...', meta: {'language': 'en'}),
 Document(id=d4d878288efba5e28a43ae0195e43dadd0298fe36d3d9b3075c5c5120d27763e, content: 'It is very central and appartement has a nice appearance (even though a lot IKEA stuff), *W A R N I ...', meta: {'language': 'en'})]

In [10]:
de_document_store.filter_documents()

[Document(id=d565bf734fac0a7e8eebfde8f67ec7f98c485d4b8c3e959249e272dabdbd51a9, content: 'Durchgängig Lärm, weil direkt an der Partymeile; schmutziges Geschirr; unvollständige Küchenausstatt...', meta: {'language': 'de'}),
 Document(id=3f34d6e83d3ddf1ddf811efe2ad8a506906353a2a7f5bbf8d4e90ef5f173d693, content: 'Das Personal ist sehr zuvorkommend! Über WhatsApp war man im guten Kontakt und konnte alles erfragen...', meta: {'language': 'de'}),
 Document(id=89a1da2e53f3987f9cd79405223b118b44beff2941d345bce86dd2502398fe72, content: 'Die Lage war sehr Zentral und man konnte alles sehenswertes zu Fuß erreichen. Wer am Wochenende nach...', meta: {'language': 'de'})]

In [11]:
indexing_pipeline.draw('pipeline.png')